<a href="https://colab.research.google.com/github/DOOMNOVA/Semantic-Textual-Similarity-STS-/blob/main/Semi_supervised_pseudo_labels_PART_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"This file includes semi-supervised pseudo-labelling of the given unlabelled dataset with a pretrained semantic textual similarity model"


"install relevant libraries"
# run cell to install packages
!pip install datasets
!pip install transformers[torch]
#!pip install accelerate -U
#!pip install seqeval
#!pip install evaluate
#!pip install pipreqs
!pip install -U "huggingface_hub[cli]"
#!pip install scikit-learn seaborn matplotlib
!pip install -U sentence-transformers


In [ ]:
"login to hugging face to upload model to hub later - uncomment code if needed"
from huggingface_hub import notebook_login
notebook_login(True)



In [3]:
import pandas as pd
import torch
from google.colab import files
import io
from sentence_transformers import SentenceTransformer, util
uploaded = files.upload()







Saving DataNeuron_Text_Similarity.csv to DataNeuron_Text_Similarity.csv


In [ ]:
"Generate pseudo labels using a pretrained model"

df_unlabelled_train = pd.read_csv(io.BytesIO(uploaded['DataNeuron_Text_Similarity.csv']))
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Define a function to calculate cosine similarity for a pair of sentences
def calculate_cosine_similarity(text1, text2):
    embedding1 = model.encode(text1, convert_to_tensor=True)
    embedding2 = model.encode(text2, convert_to_tensor=True)
    similarity_score = util.cos_sim(embedding1, embedding2)
    similarity_score_normalized = (similarity_score.item() + 1) / 2

    return similarity_score_normalized  # Extract the scalar value from the tensor



In [ ]:
# Apply the function to each row in the DataFrame and add a new column for the similarity
df_unlabelled_train['cosine_similarity'] = df_unlabelled_train.apply(lambda row: calculate_cosine_similarity(row['text1'], row['text2']), axis=1)





In [49]:
"New dataset with the pseudo labels"
df_unlabelled_train

,text1,text2,cosine_similarity
0,broadband challenges tv viewing the number of ...,gardener wins double in glasgow britain s jaso...,0.517517
1,rap boss arrested over drug find rap mogul mar...,amnesty chief laments war failure the lack of ...,0.592068
2,player burn-out worries robinson england coach...,hanks greeted at wintry premiere hollywood sta...,0.539645
3,hearts of oak 3-2 cotonsport hearts of oak set...,redford s vision of sundance despite sporting ...,0.445197
4,sir paul rocks super bowl crowds sir paul mcca...,mauresmo opens with victory in la amelie maure...,0.527641
...,...,...,...
2995,uk directors guild nominees named martin scors...,steel firm to cut 45 000 jobs mittal steel ...,0.530638
2996,u2 to play at grammy awards show irish rock ba...,israel looks to us for bank chief israel has a...,0.494892
2997,pountney handed ban and fine northampton coach...,india and iran in gas export deal india has si...,0.457765
2998,belle named best scottish band belle & sebas...,mido makes third apology ahmed mido hossam h...,0.478333


94